In [1]:
import pandas as pd

In [2]:
!pip install tensorflow_datasets
import tensorflow_datasets as tfds

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
     |████████████████████████████████| 3.6 MB 24.3 MB/s 
     |████████████████████████████████| 81 kB 102.4 MB/s 
     |████████████████████████████████| 44 kB 76.8 MB/s 
     |████████████████████████████████| 100 kB 105.9 MB/s 
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=13ef29cfc2087ab0cbf606c764f89954f3d22907854bed53aed8f443ed0900bd
  Stored in directory: /home/ubuntu/.cache/pip/wheels/cb/32/4f/8ad99d466e2fe5a68d47b0db9aeeb5b04c62a09f50becdbc92
Successfully built promise
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-metadata 0.25.0 requires absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.


In [3]:
import tensorflow as tf
import numpy as np
import math

In [4]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)

Shuffling and writing examples to /home/ubuntu/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteWTH64F/ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to /home/ubuntu/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteWTH64F/ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to /home/ubuntu/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteWTH64F/ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to /home/ubuntu/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [77]:
train_examples, val_examples, test_examples = examples['train'], examples['validation'], examples['test']

In [14]:
print(type(examples))

<class 'dict'>


In [78]:
pt = [x.numpy().decode() for x,y in train_examples]
eng = [y.numpy().decode() for x,y in train_examples]
pt_val = [x.numpy().decode() for x,y in val_examples]
eng_val = [y.numpy().decode() for x,y in val_examples]

In [34]:
train_data = pt + eng + pt_val + eng_val
f = open("train.txt", 'w')
for i in range(len(train_data)):
    data =train_data[i] + "\n"
    f.write(data)
f.close()

In [30]:
train_data = pt + eng + pt_val + eng_val
train_data[:2]

['e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .',
 'mas e se estes fatores fossem ativos ?']

In [33]:
print(pt[0])
print(pt[1])
print(eng[0])

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
mas e se estes fatores fossem ativos ?
and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


In [35]:
!pip install sentencepiece

Looking in indexes: http://ftp.daumkakao.com/pypi/simple


In [36]:
# sentencepiece를 활용하기
templates = '--input={} --pad_id={} --bos_id={} --eos_id={} --unk_id={} --model_prefix={} --vocab_size={} --character_coverage={} --model_type={}'


train_input_file = 'train.txt' # train.txt에서 문장 불러 오기
pad_id = 0
vocab_size = 40000 # 최대 단어 수는 40000으로 한다
prefix = "gpt2_spm" # sentencepiece가 gpt2_spm.model gpt2_spm.vocab으로 저장된다.
unk_id = 1 # 모르는 단어 [UNK]의 인덱스
bos_id = 2 # Beginning of sentence, 문장의 시작을 알려주는 인덱스
eos_id = 3 # End of sentecnce, 문장의 끝을 알려주는 인덱스
character_coverage = 1.0 # 한국어는 0.9정도가 좋음.(단어 하나가 함의하는게 클수록 작게 준다)
model_type = 'bpe' # 토크나이징에 bpe 기법을 사용 예정

cmd = templates.format(train_input_file,
                       pad_id,
                       bos_id,
                       eos_id,
                       unk_id,
                       prefix,
                       vocab_size,
                       character_coverage,
                       model_type)
print(cmd)
import sentencepiece as spm

spm.SentencePieceTrainer.Train(cmd) # sentencepiece 모델을 훈련시킨다

sp = spm.SentencePieceProcessor()

--input=train.txt --pad_id=0 --bos_id=2 --eos_id=3 --unk_id=1 --model_prefix=gpt2_spm --vocab_size=40000 --character_coverage=1.0 --model_type=bpe


In [37]:
sp.Load("gpt2_spm.model") # 훈련한 모델을 불러온다

True

In [38]:
with open("gpt2_spm.vocab", encoding='utf-8') as f:
    Vo = [doc.strip().split("\t") for doc in f]
word2idx = {w[0] : i for i, w in enumerate(Vo)}

In [ ]:
print(word2idx)

In [40]:
print("총 단어 수", sp.get_piece_size())
# 토큰화
print("포르투갈어", pt[0])
print("영어", eng[0])
print("포르투갈어 인덱스 부여", sp.EncodeAsIds(pt[0]))
print("인덱스를 토큰으로(포르투갈어)", sp.EncodeAsPieces(pt[0]))
print("인덱스를 토큰으로(영어)", sp.EncodeAsPieces(eng[0]))
#토큰 -> 문자열
print("토큰을 원래 문장으로", sp.DecodePieces(sp.EncodeAsPieces(eng[0])))
print("시작단어",sp.id_to_piece(2), "끝단어",sp.id_to_piece(3))

총 단어 수 40000
포르투갈어 e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
영어 and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
포르투갈어 인덱스 부여 [8, 347, 883, 168, 4, 3646, 6, 13430, 4, 2292, 9068, 138, 7232, 6, 56, 89, 4, 32173, 260, 7]
인덱스를 토큰으로(포르투갈어) ['▁e', '▁quando', '▁melhor', 'amos', '▁a', '▁procura', '▁,', '▁tiramos', '▁a', '▁única', '▁vantagem', '▁da', '▁impressão', '▁,', '▁que', '▁é', '▁a', '▁serendip', 'idade', '▁.']
인덱스를 토큰으로(영어) ['▁and', '▁when', '▁you', '▁improve', '▁search', 'ability', '▁,', '▁you', '▁actually', '▁take', '▁away', '▁the', '▁one', '▁advantage', '▁of', '▁print', '▁,', '▁which', '▁is', '▁serendipity', '▁.']
토큰을 원래 문장으로 and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
시작단어 <s> 끝단어 </s>


In [42]:
print(sp.piece_to_id("<s>"))


2


In [79]:
def encode(lang1, lang2):
  # 포르투갈어 + <영어 시작> + 영어 + <영어 끝>
    lang = sp.EncodeAsIds(lang1) + [sp.piece_to_id("<s>")] + sp.EncodeAsIds(lang2) + [sp.piece_to_id("</s>")]# 포르투갈 어를 인코딩 할 때 시작 단어를 의미하는 숫자와, 끝 단어를 의미하는 숫자가 붙음
  # 텐서플로우에서 활용할 수 있는 텐서로 바꿔준다
    lang = tf.convert_to_tensor(lang)
  # 텐서의 모양을 정해줘야 함, 문장의 길이는 변하므로 [None]으로
    lang.set_shape([None])
    return lang

In [80]:
# texts = 트레인 데이터 셋
# texts_val = valid 데이터 셋
texts = [encode(x,y) for x,y in zip(pt, eng)]
texts_val = [encode(x,y) for x,y in zip(pt_val, eng_val)]

In [81]:
MAX_LENGTH = 100
BUFFER_SIZE = 60000
BATCH_SIZE = 64

# 만약 문장이 인코딩 되었을 때, 인코딩 된 길이가 100을 초과하면 데이터에서 배제하고자 하는 필터 함수임
def filter_max_length(x, max_length=MAX_LENGTH):
    return tf.size(x) <= max_length

In [125]:
print(texts[0])

tf.Tensor(
[    8   347   883   168     4  3646     6 13430     4  2292  9068   138
  7232     6    56    89     4 32173   260     7     2    58   358    90
  5557  4278  3129     6    90   563   777  1636    32   277  8868    64
  5182     6   534    80 39049     7     3], shape=(43,), dtype=int32)


In [135]:
!pip show tensorflow

Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages
Requires: scipy, tensorflow-estimator, gast, wrapt, absl-py, keras-preprocessing, opt-einsum, tensorboard, h5py, google-pasta, termcolor, six, numpy, wheel, grpcio, protobuf, astunparse
Required-by: tensorflow-serving-api


In [136]:
print(type(generator))
print(generator)

<class 'function'>
<function generator at 0x7f08d30ad510>


In [139]:
# Train 데이터(texts)를 generator를 활용하여 텐서플로우 dataset을 만듦
def generator():
    for el in texts:
        yield el
dataset = tf.data.Dataset.from_generator(generator,
                                         output_types= tf.int64,output_shapes=tf.TensorShape([None,]))
dataset = dataset.filter(filter_max_length) # 길이가 100 초과하면 날리기
dataset = dataset.cache() # cache를 활용해서 데이터를 로드할 때 빠른 처리를 기대해 봄
dataset = dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=[None]) # padded_batch 불러오기
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE) # 데이터가 훈련되는 사이에, 데이터를 미리 준비해 놓음

[array([    8,   347,   883,   168,     4,  3646,     6, 13430,     4,
        2292,  9068,   138,  7232,     6,    56,    89,     4, 32173,
         260,     7,     2,    58,   358,    90,  5557,  4278,  3129,
           6,    90,   563,   777,  1636,    32,   277,  8868,    64,
        5182,     6,   534,    80, 39049,     7,     3]), array([  174,     8,    96,   981,  9414,  5329, 12064,   121,     2,
         180,   190,   306,    84,   439,  7192,   121,     3]), array([  174,   524,   126,  1882,     4,  9094,    39,   144,  6975,
           7,     2,   180,   169,   547,    23, 39911, 39892,  1231,
         111,  9071,     7,     3])]


In [83]:
# valid 데이터 또한 텐서플로우 dataset으로 만들기
def val_generator():
    for el in texts_val:
        yield el
val_dataset = tf.data.Dataset.from_generator(val_generator,
                                           output_types= tf.int64)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(BATCH_SIZE, padded_shapes=[None])

In [84]:
# 단어 임베딩 정의
class EmbeddingLayer(tf.keras.layers.Layer):

    def __init__(self, vocab_size, embedding_size, initializer=None, stddev=0.02, mean=0.0):
        super().__init__(self)
        self.vocab_size = vocab_size # 훈련할 데이터의 vocab size
        self.embedding_size = embedding_size # 단어가 고차원으로 임베딩되는 차원
        self.stddev = stddev # 초기 임베딩 계수들의 표준편차
        self.mean = mean # 초기 임베딩 계수들의 평균
        self.initializer = initializer
        if self.initializer is None:
            self.initializer = tf.random_normal_initializer(mean=self.mean,
                                                      stddev = self.stddev) # N~(0, 2^2)의 분포로 초기 계수를 정규화
    def build(self, input_shape):
        with tf.name_scope("embedding_weights"):
            self.embedding_weights = self.add_weight(
              "weights",
              shape=[self.vocab_size, self.embedding_size],
              dtype="float32",
              initializer=self.initializer) # [vocab_size, embedding_size]의 계수 정의
  
    def call(self, inputs, mode="embedding", scale=False):
        if mode == "embedding":
            return self.embedding(inputs, scale=scale) # embedding 모드이면, 인풋 단어를 임베딩 할 때 사용한다
        elif mode == "projection":
            return self.projection(inputs) # projection 모드이면, 최종 출력 값에 임베딩 계수들을 다시 곱해준다
        else:
            raise ValueError("mode {} is not valid.".format(mode))
  
    def embedding(self, inputs, scale=False):
        with tf.name_scope("embedding"):
            mask = tf.cast(tf.not_equal(inputs, 0), tf.float32) # 마스크를 만들어서 패딩인 부분을 0으로 마스크
            inputs = tf.cast(inputs, tf.int32) # 인풋 값을 텐서로 바꿔줌, input_shape -> [batch_size, length]

        embeddings = tf.nn.embedding_lookup(self.embedding_weights, inputs) # 임베딩 룩업 테이블을 곱해서 단어를 임베딩
      # embeddings_shape -> [batch_size, length, embedding_size]
      # mask size is converted to [batch_size, length, 1]
        embeddings *= tf.expand_dims(mask, -1)

        if scale:
            embeddings *= self.embedding_size ** 0.5

        return embeddings
  
  # GPT2 최종 출력 값에, 임베딩 벡터를 다시 곱해줌
    def projection(self, inputs):
        with tf.name_scope("output_layer"):
            batch_size = tf.shape(inputs)[0]
            seq_len = tf.shape(inputs)[1]
      # 출력 값의 input shape [batch_size, seq_len, d_model] 이다
      # seq_len : 문장 길이, d_model = 단어 embedding 차원
            h_flat = tf.reshape(inputs, [-1, self.embedding_size])
      # h_flat shape = [batch_size*seq_len, d_model]
            logits = tf.matmul(h_flat, self.embedding_weights, transpose_b=True)
      # logits shape = [batch_size*seq_len, vocab_size]
            return tf.reshape(logits, [batch_size, seq_len, self.vocab_size])
      # final output shape = [batch_size, seq_len, vocab_size]

# 포지셔널 임베딩 정의
class PositionEmbeddingLayer(tf.keras.layers.Layer):

    def __init__(self, position_seq, pos_embedding_size, trainable=True, stddev=0.02, mean=0.0):
        tf.keras.layers.Layer.__init__(self, name="PositionEmbeddingLayer")

        self.position_seq = position_seq # 보통 적당히 큰 숫자(10000) 정도로 입력한다음 문장의 길이에 맞게 자름
        self.hidden_size = pos_embedding_size
        self.trainable = trainable
        self.stddev = stddev
        self.mean = mean

        if trainable:
            self.position_embedding = EmbeddingLayer(self.position_seq, self.hidden_size, stddev=self.stddev, mean=self.mean)

    def call(self, inputs, start=1):
        with tf.name_scope("pos_embedding"):
            if self.trainable:
                batch_size = tf.shape(inputs)[0]
                batch_seq = tf.shape(inputs)[1]

                positions = tf.reshape(tf.tile(tf.range(start, batch_seq + start), [batch_size]), [batch_size, batch_seq])
                # positions는 직접 해보면 좋음
                positions = tf.cast(positions, tf.int32)
                position_mask = tf.cast(tf.not_equal(inputs, 0), tf.int32)
                positions *= position_mask

                return self.position_embedding(positions)

            else:
                return self.get_position_sinusoid(self.position_seq, self.hidden_size) # 트랜스포머에서 한 것과 유사함. 생략해도 좋음

    @staticmethod
    def get_position_sinusoid(seq_len, hidden_size, min_timescale=1.0, max_timescale=1.0e4):
        position = tf.cast(tf.range(seq_len), tf.float32)
        num_timescales = hidden_size // 2
        log_timescale_increment = (math.log(float(max_timescale) / float(min_timescale)) / (tf.cast(num_timescales, tf.float32) - 1))
        inv_timescales = min_timescale * tf.exp(tf.cast(tf.range(num_timescales), tf.float32) * -log_timescale_increment)
        scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
        signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
        return signal

In [85]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, att_dropout=0.1, residual_dropout=0.1, scale=True):
        super().__init__(self, name="MultiHeadAttention")
        self.num_heads = num_heads # 병렬로 나눠서 처리할 숫자
        self.d_model = d_model # 임베딩 차원
        self.att_dropout = att_dropout # 어텐션 드롭아웃
        self.residual_dropout = residual_dropout # 레지듀얼 드롭아웃
        self.scale = scale # 스케일링 할지 말지 (Query * Value의 값을 SQRT(임베딩차원) 으로 나눔)

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.c_attn = tf.keras.layers.Dense(self.d_model * 3) # 출력 차원을 3배로 한 다음에 q, k, v로 나눔 
        self.c_proj = tf.keras.layers.Dense(self.d_model)

    def split_heads(self, x):
        batch_size = tf.shape(x)[0]
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0,2,1,3])

    def multihead_attention(self, q, k, v, training, mask=None):
    # depth : d_model // seq_len
    # q_shape : [batch_size, num_heads, seq_q, depth]
    # k_shape : [batch_size, num_heads, seq_k, depth]
    # v_shape : [batch_size, num_heads, seq_v, depth]

        matmul_qk = tf.matmul(q, k, transpose_b=True) # shape -> [batch_size, num_heads, seq_q, seq_k]
        if self.scale:
            dk = tf.cast(tf.shape(k)[-1], tf.float32)
            matmul_qk = matmul_qk / tf.math.sqrt(dk)

        if mask is not None:
            matmul_qk += (mask * -1e9) # 마스크를 곱해준다

        attention_weights = tf.nn.softmax(matmul_qk, axis=-1) # shape -> [batch_size, num_heads, seq_q, seq_k]

        if training:
            attention_weights = tf.nn.dropout(attention_weights, rate=self.att_dropout, name="attn_dropout")
        output = tf.matmul(attention_weights, v) # output shape -> [batch_size, num_heads, seq_q, depth]

        return output, attention_weights

    def merge_heads(self, x):
        batch_size = tf.shape(x)[0]
        # x shape -> [batch_size, num_heads, seq_q, depth]
        x = tf.transpose(x, perm=[0,2,1,3])
        # x shape -> [batch_size, seq_q, num_heads, depth]
        merged = tf.reshape(x, (batch_size, -1, self.d_model))
        # merged shape -> [batch_size, seq_q, d_model]
        return merged

    def call(self, x, mask=None, past_layer=None, training=True):
        # input x shape -> [batch_size, seq_len, d_model]
        x = self.c_attn(x)
        # x shape -> [batch_size, seq_len, d_model * 3]
        query, key, value = tf.split(x, 3, axis=2)
        # query_shape -> [batch_size, seq_len, d_model]
        # key_shape -> [batch_size, seq_len, d_model]
        # value_shape -> [batch_size, seq_len, d_model]

        query = self.split_heads(query)
        key = self.split_heads(key)
        value = self.split_heads(value)

        # query_shape -> [batch_size, num_heads, seq_len, d_model // num_heads]
        # key_shape -> [batch_size, num_heads, seq_len, d_model // num_heads]
        # value_shape -> [batch_size, num_heads, seq_len, d_model // num_heads]

        # past -> 과거 자료 활용
        # 예를 들어 "I have my chair repaired to study NLP"라는 문장이 있다고 하자
        # 마지막 단어를 예측할 때 NLP라는 단어를 예측한다고 하면
        # I have my chair repaired to study라는 문장을 GPT에 넣으면 NLP를 예측하게 된다.
        # 그런데 i have my chair repaired to 를 활용해서 study 라는 단어도 예측했을 것이다.
        # 따라서 i have my chair repaired to 를 GPT에 넣었을 때의 출력을 재활용하면
        # study라는 단어만 신경망을 거치면 되므로 예측 속도가 상당히 빨라지게 된다.

        if past_layer is not None:
          # past_layer의 shape은 [batch_size, 2, num_heads, seq_past_len, d_model // num_heads]
            past_key, past_value = tf.unstack(past_layer, axis=1)
          # past_key_shape = [batch_size, num_heads, seq_past_len, d_model // num_heads]
          # past_value_shape = [batch_size, num_heads, seq_past_len, d_model // num_heads]
            key = tf.concat([past_key, key], axis=-2)

            value = tf.concat([past_value, value], axis=-2)
          # key_value_shape = [batch_size, num_heads, seq_past_len + seq_len, d_model // num_heads]
          # value_value_shape = [batch_size, num_heads, seq_past_len + seq_len, d_model // num_heads]
        present = tf.stack([key, value], axis=1)
        # present shape -> [batch_size, 2, num_heads, seq_past_len + seq_len, d_model // num_heads]
        scaled_attention, attention_weights = self.multihead_attention(query, key, value, training, mask)
        #scaled_attention shape -> [batch_size, num_heads, seq_len, d_model // num_heads]
        #attention_weights_shape -> [batch_size, num_heads, seq_len, seq_past_len + seq_len, d_model // num_heads]
        concat_attention = self.merge_heads(scaled_attention)
        # concat_attention shape -> [batch_size, seq_len, d_model]
        output = self.c_proj(concat_attention)

        if training:
            output = tf.nn.dropout(output, rate=self.residual_dropout, name="resid_dropout")
        # output shape -> [batch_size, seq_len, d_model]
        return output, present

In [86]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, hidden_size, dff, dropout_rate=0.1, activation=tf.nn.relu):
        super().__init__(self, name="FeedForward")
        self.hidden_size = hidden_size # d_model
        self.dff = dff # middle dimension
        self.activation = activation # relu 
        self.dropout_rate = dropout_rate

        self.dense_layer = tf.keras.layers.Dense(self.dff)
        self.output_dense_layer = tf.keras.layers.Dense(self.hidden_size)

    def call(self, x, training=False):
        # input x shape -> [batch_size, seq_len, d_model]
        output = self.dense_layer(x)
        # shape -> [batch_size, seq_len, dff]
        output = self.activation(output)
        output = self.output_dense_layer(output)
        # shape -> [batch_size, seq_len, d_model]
        if training:
            output = tf.nn.dropout(output, rate=self.dropout_rate, name="feed_forward_dropout")

        return output

In [87]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, i=0, dropout_rate = 0.1):
        super().__init__(self, name="Decode_{}".format(i))
        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff
        self.dropout_rate = dropout_rate

        self.mha = MultiHeadAttention(self.d_model, self.num_heads, self.dropout_rate)
        self.feed_forward = FeedForward(self.d_model, self.dff, self.dropout_rate)
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training, mask, past=None):
        # input x -> [batch_size, seq_len, d_model]
        out, present = self.mha(self.layer_norm1(x), mask=mask, past_layer=past, training=training)
        # out -> [batch_size, seq_len, d_model]
        # present shape -> [batch_size, 2, num_heads, seq_past_len + seq_len, d_model // num_heads]
        with tf.name_scope("residual_conn_1"):
            x = x + out

        out = self.feed_forward(self.layer_norm2(x), training=training)
        with tf.name_scope("residual_conn_2"):
            x = x + out
        return x, present

In [88]:
class OutputLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, proj_weights=None, kernel_initializer=None):
        super().__init__(self, name="OutputLayer")
        self.proj_weights = proj_weights
        self.output_dim = output_dim
        self.layer_weights = None
        self.kernel_initializer = kernel_initializer
        self.layer_weights = tf.keras.layers.Dense(self.output_dim)
    def call(self, x):
    # input x shape -> [batch_size, seq_len, d_model]
        batch, sequence, d_model = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[-1]

        if self.proj_weights is None:

            out = self.layer_weights(x)
          # out shape -> [batch_size, seq_len, vocab_size]
        else:
            out = self.proj_weights(x)
          # out shape -> [batch_size, seq_len, vocab_size]
        out = tf.reshape(out, [batch, sequence, self.output_dim])
        return out

In [89]:
# 유틸리티 함수 정의
def shape_as_list_2(x):
    return [int(i) for i in tf.shape(x)]


def gelu(x):
    with tf.name_scope("gelu"):
        cdf = 0.5 * (1.0 + tf.tanh(
            (np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
        return x * cdf


def get_padding_mask(seq):
    with tf.name_scope("Padding_Mask"):
        seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

        # add extra dimensions to add the padding
        # to the attention logits.
        return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def attention_mask(size):
    """
    if size is 4 then it returns below matrix
       [[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]]
    """
    with tf.name_scope("attention_mask"):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
        return mask  # (seq_len, seq_len)


def create_masks(inp):
    with tf.name_scope("att_masking"):
        att_mask = attention_mask(tf.shape(inp)[1])
        padding_mask = get_padding_mask(inp)
        mask = tf.maximum(padding_mask, att_mask)

        return mask

In [90]:
class GPT2(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, max_seq_len, vocab_size,\
               rev_embedding_projection=True):
        super().__init__(self, name="GPT2_TOTAL")

        self.rev_embedding_projection = rev_embedding_projection # 임베딩과 출력증을 묶을지
        self.num_layers = num_layers # 디코더 층
        self.num_heads = num_heads # 멀티 헤드 어텐션 때 헤드 숫자
        self.dff = dff # 중간층 차원
        self.max_seq_len = max_seq_len # position embedding시 임베딩할 최대 넓이
        self.vocab_size = vocab_size # 단어의 수
        self.d_model = d_model # 단어가 임베딩 될 때 차원

        self.embedding = EmbeddingLayer(self.vocab_size, self.d_model) # 단어 임베딩
        self.pos_embedding = PositionEmbeddingLayer(self.max_seq_len, self.d_model) # 위치 임베딩
        self.decoder_layers = [DecoderLayer(self.d_model, self.num_heads, self.dff, i) for i in range(self.num_layers)] # 디코더를 num_layers 만큼 쌓음

        if not self.rev_embedding_projection: # 만약 임베딩과 출력증을 같이 묶지 않으면
            self.output_layer = OutputLayer(self.vocab_size)

        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=True, past=None):
    
        x = tf.cast(x, tf.int32) # [batch_size, seq_len]
        batch, sequence = tf.shape(x)[0], tf.shape(x)[1]

        if past is None:
            pasts = [None] * self.num_layers
        else:
            pasts = past
    
        assert len(pasts) == self.num_layers

        att_mask = create_masks(x) # 어텐션 마스크

        past_length = 1 if past is None else tf.shape(past)[-2]

        with tf.name_scope("embeddings"):
            embedded_x = self.embedding(x)
            hidden_states = embedded_x + self.pos_embedding(x, start=past_length) # hidden_states -> [batch_size, seq_len, d_model]
    
        presents = []
        for decoder_layer, past in zip(self.decoder_layers, pasts):
            hidden_states, present = decoder_layer(hidden_states, training, att_mask, past=past)
      # hidden_states -> [batch_size, seq_len, d_model]
      # present shape -> [batch_size, 2, num_heads, seq_past_len + seq_len, d_model // num_heads]
            presents.append(present)

        hidden_states = self.layer_norm(hidden_states)

        if self.rev_embedding_projection: # 만약 임베딩과 출력층을 결합한다면
            logits = self.embedding(hidden_states, mode="projection")
          # shape -> [batch_size, seq_len, vocab_size]
        else:
            logits = self.output_layer(hidden_states)
          # shape -> [batch_size, seq_len, vocab_size]
        return logits, presents

In [91]:
num_layers = 4 # 디코더 층
d_model = 256 # 단어 임베딩 차원
num_heads = 4 # 어텐션 크기
dff = 512 # 중간 차원
max_seq_len = 10000 # 위치 임베딩의 길이
vocab_size = 40000 # 어휘 숫자

In [92]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [93]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none') #from_logits=True로 하면 Dense 이후 softmax layer 값 출력

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0)) # 예를 들어서 실제 자료(0은 패딩)가 [1,2,3,4,5,0,0,0,0,0] 이라면 [0,0,0,0,0,1,1,1,1,1]로 바꿔 줌
                                                     # 이후 tf.math.logical_not을 활용해서 [True,True,True,True,True,False,False,False,False,False]으로 바꿔 줌
    loss_ = loss_object(real, pred) # loss_는 패딩을 고려하지 않은 loss 값

    mask = tf.cast(mask, dtype=loss_.dtype) # [True,True,True,True,True,False,False,False,False,False]를 [1,1,1,1,1,0,0,0,0,0] 으로 바꿔 줌
    loss_ *= mask # loss에 mask를 곱해서, 패딩인 부분은 0처리 해줌

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


# 패딩이 아닌 부분만을 가지고 정확도를 측정함
def get_padded_accuracy(labels, logits):
    with tf.name_scope("padded_accuracy"):
        weights = tf.cast(tf.not_equal(labels, 0), tf.float32)

        outputs = tf.cast(tf.argmax(logits, axis=-1), tf.int32)
        padded_labels = tf.cast(labels, tf.int32)

        nonpad_seq = tf.math.count_nonzero(weights, dtype=tf.dtypes.float32, )
        acc = tf.cast(tf.equal(outputs, padded_labels), tf.float32)

        accuracy = tf.reduce_sum(tf.cast(acc * weights, tf.float32)) / nonpad_seq

    return tf.cast(accuracy, tf.float32)

In [94]:
# 저장할 체크포인트 지정
checkpoint_path = "./"

ckpt = tf.train.Checkpoint(gpt2=gpt2,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [95]:
x = np.random.uniform(size=(1,40)).astype('float32')
print(x)
gpt2 = GPT2(12, 512, 8, 2048, 400, 50000)
a, b = gpt2(x)
print(a)

[[0.93320173 0.7463982  0.94402003 0.6174087  0.94426596 0.21401747
  0.11069497 0.9469942  0.48068184 0.9681034  0.41313925 0.29743963
  0.8416083  0.6202419  0.5824169  0.22635405 0.11839538 0.2094927
  0.9117539  0.5161936  0.10331562 0.54510826 0.5257167  0.16830607
  0.59243035 0.6855068  0.07447074 0.11094531 0.7689338  0.44184098
  0.3474238  0.15635541 0.16046679 0.278723   0.0932311  0.2217316
  0.9279012  0.76593196 0.35014048 0.91539276]]
tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 40, 50000), dtype=float32)


In [96]:
gpt2 = GPT2(num_layers = num_layers, d_model=d_model, num_heads=num_heads, dff=dff, max_seq_len=max_seq_len, vocab_size=vocab_size)
train_step_signature = [tf.TensorSpec(shape=(None, None), dtype=tf.int64)]

# tf.function을 사용하면 그래프를 미리 컴파일 하기 때문에 속도가 상당히 빠름
# 같은 GPU여도 케라스에 비해서 체감상 7~8배 정도의 차이가 나는 것 같음
@tf.function(input_signature=train_step_signature)
def train_step(inputs, grad_clip=True, clip_value=2.5):
    inp = inputs[:, :-1]
    tar = inputs[:, 1:]

    with tf.GradientTape() as tape:
        predictions, _ = gpt2(inp, True)
        loss = loss_function(tar, predictions)

    gradients = tape.gradient(loss, gpt2.trainable_variables)  
    if grad_clip:
        gradients = [(tf.clip_by_value(grad, -clip_value, clip_value)) for grad in gradients]  
    optimizer.apply_gradients(zip(gradients, gpt2.trainable_variables))
  
    accuracy = get_padded_accuracy(tar, predictions)
    return loss, accuracy

In [ ]:
import time
# 20 에포크 훈련
for epoch in range(20):
    start = time.time()
  
  #train_loss.reset_states()
  #train_accuracy.reset_states()
  
  # input : 포루투갈어, tar : 영어
    for (batch, inp) in enumerate(dataset):
        loss, accuracy = train_step(inp)
    
        if batch % 50 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
                epoch + 1, batch, loss, accuracy))
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                         loss,
                                                         accuracy))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 10.6207 Accuracy 0.0000
Epoch 1 Batch 50 Loss 10.4015 Accuracy 0.0232
Epoch 1 Batch 100 Loss 9.9555 Accuracy 0.0586
Epoch 1 Batch 150 Loss 9.5209 Accuracy 0.0534
Epoch 1 Batch 200 Loss 8.8616 Accuracy 0.0590
Epoch 1 Batch 250 Loss 8.1687 Accuracy 0.0873
Epoch 1 Batch 300 Loss 7.5581 Accuracy 0.1123
Epoch 1 Batch 350 Loss 7.0891 Accuracy 0.1127
Epoch 1 Batch 400 Loss 6.6717 Accuracy 0.1382
Epoch 1 Batch 450 Loss 6.4759 Accuracy 0.1438
Epoch 1 Batch 500 Loss 6.4461 Accuracy 0.1451
Epoch 1 Batch 550 Loss 6.0227 Accuracy 0.1657
Epoch 1 Batch 600 Loss 6.1645 Accuracy 0.1805
Epoch 1 Batch 650 Loss 5.9188 Accuracy 0.1920
Epoch 1 Batch 700 Loss 5.9567 Accuracy 0.1839
Epoch 1 Batch 750 Loss 5.8723 Accuracy 0.2001
Epoch 1 Loss 5.8872 Accuracy 0.1914
Time taken for 1 epoch: 1210.4899151325226 secs

Epoch 2 Batch 0 Loss 5.6257 Accuracy 0.2032
Epoch 2 Batch 50 Loss 5.5939 Accuracy 0.2080
Epoch 2 Batch 100 Loss 5.6226 Accuracy 0.2097
Epoch 2 Batch 150 Loss 5.6012 Accuracy 0.2241